Let’s work on the following:
1. Access the CB API (with my login info), which is inside Dropbox. Download the list of companies founded in 2016 and with category ‘Artificial Intelligence’. That should be 500sh companies, we start with a small sample.
2. For each of those companies, download all the NEWS that Crunchbase has. Try to put those news into an easy format: maybe a TXT file per news, and all TXT files of a company into one folder. And remove all non-relevant components (like Ads or other text). This will make it easier if somebody in the team will have to do some reading.
3. Once this looks good, we’ll do some NLP on the text. But let’s get the "raw data" first.
4. Please upload all material on Dropbox.

In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import nltk
from nltk import *
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import sys
import codecs
import string
import random
import re

import requests
import urllib
import bs4
from bs4 import BeautifulSoup
from readability.readability import Document # https://github.com/buriy/python-readability. Tried Goose, Newspaper (python libraries on Github). Bad results.
from http.cookiejar import CookieJar # 

import pycrunchbase
cb = pycrunchbase.CrunchBase('662e263576fe3e4ea5991edfbcfb9883')

/Users/ksun/Downloads/venv3/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
# df = pd.DataFrame(columns=["permalink"])

# url = 'https://api.crunchbase.com/v3.1/organizations/?user_key=662e263576fe3e4ea5991edfbcfb9883'
# # location given here 
# location = "philadephia"

# # defining a params dict for the parameters to be sent to the API 
# PARAMS = {'address':location} 

# while True:
#     try:
#         response = requests.get(url, PARAMS)
#         json = response.json()
#         for c in json['data']['items']:
#             permalink=c['properties']['permalink']
#             if(permalink!=None):
#                 categories_json = requests.get("https://api.crunchbase.com/v3.1/organizations/" 
#                                 + permalink+"/categories?user_key=662e263576fe3e4ea5991edfbcfb9883", PARAMS).json()
#                 for cat in categories_json['data']['items']:
#                     if(cat["properties"]["category_groups"]!=None):
#                         if("Artificial Intelligence" in cat["properties"]["category_groups"]):
#                             print(permalink)
#                             df = df.append({'permalink' : permalink} , ignore_index=True)
#         url = json['data']['paging']['next_page_url'] + "&user_key=662e263576fe3e4ea5991edfbcfb9883"
#         print(url)
#     except:
#         print("error: " + url)
        
# df

df = pd.read_csv('csv_export/organizations.csv')
print(df.columns)
df

/Users/ksun/Downloads/venv3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['company_name', 'roles', 'permalink', 'domain', 'homepage_url',
       'country_code', 'state_code', 'region', 'city', 'address', 'status',
       'short_description', 'category_list', 'category_group_list',
       'funding_rounds', 'funding_total_usd', 'founded_on', 'last_funding_on',
       'closed_on', 'employee_count', 'email', 'phone', 'facebook_url',
       'linkedin_url', 'cb_url', 'logo_url', 'twitter_url', 'aliases', 'uuid',
       'created_at', 'updated_at', 'primary_role', 'type'],
      dtype='object')


,company_name,roles,permalink,domain,homepage_url,country_code,state_code,region,city,address,...,linkedin_url,cb_url,logo_url,twitter_url,aliases,uuid,created_at,updated_at,primary_role,type
0,Grown Ocean,company,/organization/grown-ocean,grownocean.com,http://www.grownocean.com,USA,NY,New York City,New York,"79 Madison Avenue, 2nd Floor",...,https://www.linkedin.com/company/6440106?trk=t...,https://www.crunchbase.com/organization/grown-...,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/GrownOceanApp,NaN,eea6f58a-b728-d416-7e38-6ca5bad23e2d,2016-04-26 20:25:23,2018-10-26 22:27:32,company,organization
1,Intel,"company,investor",/organization/intel,intel.com,http://www.intel.com/,USA,CA,SF Bay Area,Santa Clara,2200 Mission College Blvd,...,https://www.linkedin.com/company/intel-corpora...,https://www.crunchbase.com/organization/intel,https://crunchbase-production-res.cloudinary.c...,http://twitter.com/intel,N M Electronics,1e4f199c-363b-451b-a164-f94571075ee5,2008-02-06 06:28:14,2018-12-11 10:52:30,company,organization
2,Intercomp,company,/organization/intercomp,intercompcompany.com,http://www.intercompcompany.com/,USA,OH,Cleveland,Medina,3839 County Road 116,...,https://www.linkedin.com/company/906845,https://www.crunchbase.com/organization/intercomp,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,6681b1b0-0cea-6a4a-820d-60b15793fa66,2015-07-26 19:09:56,2018-04-17 12:08:01,company,organization
3,Gruppo Editoriale L'Espresso Spa,company,/organization/gruppo-editoriale-lespresso-spa,gruppoespresso.it,http://www.gruppoespresso.it/en/nc.html,ITA,NaN,Rome,Rome,"Gruppo Editoriale L’Espresso Spa, Via Cristofo...",...,https://www.linkedin.com/company/952199,https://www.crunchbase.com/organization/gruppo...,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,e74463f7-94a8-ab7b-ba31-f4cd57a16570,2012-04-24 19:27:30,2018-08-07 12:49:54,company,organization
4,Foster Yeoman Ltd,company,/organization/foster-yeoman-ltd,NaN,NaN,GBR,NaN,GBR - Other,Frome,Marston Bigot Marston House,...,NaN,https://www.crunchbase.com/organization/foster...,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,3d40d3e3-8452-4675-9a75-b3bf6f44fc65,2019-02-20 14:55:04,2019-02-20 14:55:04,company,organization
5,LSI Logic,"company,investor",/organization/lsi,lsi.com,http://www.lsi.com,USA,CA,SF Bay Area,Milpitas,1621 Barber Lane,...,http://www.linkedin.com/company/lsi-corporation,https://www.crunchbase.com/organization/lsi,https://crunchbase-production-res.cloudinary.c...,http://twitter.com/LSICorporation,LSI Logic,aa320353-a1eb-5368-8a51-f39e27954f90,2008-05-23 04:19:43,2018-10-26 22:39:57,company,organization
6,Microsoft,"company,investor",/organization/microsoft,microsoft.com,http://www.microsoft.com,USA,WA,Seattle,Redmond,1 Microsoft Way,...,http://www.linkedin.com/company/microsoft,https://www.crunchbase.com/organization/microsoft,https://crunchbase-production-res.cloudinary.c...,http://twitter.com/Microsoft,msn,fd80725f-53fc-7009-9878-aeecf1e9ffbb,2014-05-12 12:00:58,2019-02-01 16:07:42,company,organization
7,3Com,"company,investor",/organization/3com,3com.com,http://www.3com.com,USA,MA,Boston,Marlborough,350 Campus Dr,...,http://in.linkedin.com/company/3com,https://www.crunchbase.com/organization/3com,https://crunchbase-production-res.cloudinary.c...,https://www.twitter.com/hpe,NaN,5e645e6c-d435-fd14-f82f-efa7a300cc86,2008-09-08 11:21:20,2018-08-21 09:12:34,company,organization
8,Compaq,"company,investor",/organization/compaq,compaq.com,http://www.compaq.com,USA,CA,SF Bay Area,Palo Alto,3000 Hanover Street,...,https://www.linkedin.com/company/hp,https://www.crunchbase.com/organization/compaq,https://crunchbase-production-res.cloudinary.c...,https://www.twitter.com/hpnews,NaN,10a3b2fd-b142-046b-7d8f-3b1aa4877aca,2008-08-26 04:03:18,2018-04-17 09:10:55,company,organization
9,BDS,company,/organization/bds-incorporated,NaN,NaN,USA,NC,Asheville,Lenoir,1260 Hidden Cove Lane,...,NaN,https://www.crunchbase.com/organization/b

In [3]:
ai_df = pd.DataFrame(columns = df.columns)
for i in range(len(df)):
    try:
        if("Artificial Intelligence" in df['category_group_list'][i] and '2016' in df['founded_on'][i]):
            ai_df = ai_df.append(df.loc[i])
    except:
        pass
ai_df

,company_name,roles,permalink,domain,homepage_url,country_code,state_code,region,city,address,...,linkedin_url,cb_url,logo_url,twitter_url,aliases,uuid,created_at,updated_at,primary_role,type
20093,Ubiquity.ai,company,/organization/ubiquity-ai,ubiquity.ai,http://ubiquity.ai,NaN,NaN,NaN,NaN,NaN,...,https://www.linkedin.com/company/ubiquity.ai/,https://www.crunchbase.com/organization/ubiqui...,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,aaa64a6f-2e07-4af1-b81e-dfbd362bd82f,2018-06-14 11:25:44,2018-10-27 00:21:13,company,organization
24862,Viralheat,"company,investor",/organization/viralheat,viralheat.com,http://www.viralheat.com,USA,CA,SF Bay Area,San Mateo,1510 Fashion Island Blvd. Ste. 260,...,https://www.linkedin.com/company/viralheat/,https://www.crunchbase.com/organization/viralheat,https://crunchbase-production-res.cloudinary.c...,http://twitter.com/viralheat,NaN,500ae0bc-dcb0-9d61-f228-92289c56a8ef,2009-05-25 21:49:10,2018-12-11 11:26:45,company,organization
49504,DealStart,company,/organization/dealstart,dealstart.com,https://dealstart.com,USA,NY,New York City,New York,"79 Madison Avenue, 7th Floor",...,https://www.linkedin.com/company/dealstart,https://www.crunchbase.com/organization/dealstart,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/dealstart,NaN,a7fe8d37-4ab6-71e9-4a7b-3d0bc4569eeb,2014-03-11 00:39:35,2019-01-19 03:56:17,company,organization
54071,UniQreate,company,/organization/uniqreate-limited,uniqreate.com,http://www.uniqreate.com,USA,NY,New York City,New York,"353 Lexington Avenue, Suite 1605",...,https://www.linkedin.com/company/uniqreate,https://www.crunchbase.com/organization/uniqre...,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/Uni_Qreate,"UQ,ReQon,Xtract",b85e315c-a61c-adc1-6a78-5c57300a4361,2015-09-22 07:43:15,2019-01-28 04:31:27,company,organization
57557,Really,company,/organization/really-really-inc,really.ai,https://really.ai/,USA,CA,SF Bay Area,Palo Alto,950 Page Mill Road,...,https://www.linkedin.com/company-beta/17978433/,https://www.crunchbase.com/organization/really...,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/reallyreallyio,NaN,0441a298-919d-8999-2977-88a7b144ff06,2017-03-08 04:33:08,2018-10-26 23:19:07,company,organization
59958,Flyreel,company,/organization/flyreel,flyreel.com,http://flyreel.com,USA,CO,Denver,Denver,3000 Lawrence Street,...,https://www.linkedin.com/company/2933039,https://www.crunchbase.com/organization/flyreel,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,9dad2e54-1b28-ddbb-9d5f-e4b1263deccc,2015-08-25 08:24:47,2018-10-26 22:17:06,company,organization
67958,DISCERN,company,/organization/discern,discern.com,http://www.discern.com,USA,CA,SF Bay Area,San Mateo,493 South El Camino Real,...,https://www.linkedin.com/company/discern,https://www.crunchbase.com/organization/discern,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/DISCERN_Data,NaN,491d642a-6899-fd3c-5a85-d2851ee8c89e,2012-12-14 16:07:20,2019-01-25 11:55:18,company,organization
68527,Polte Corporation,company,/organization/polte,polte.com,https://www.polte.com/,USA,TX,Dallas,Richardson,801 E. Campbell Road\nSuite 580\n,...,https://www.linkedin.com/company/polte/,https://www.crunchbase.com/organization/polte,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/PoLTECorp,Invisitrack,308560bf-494f-4ab4-8ce3-b7164450fa62,2018-04-09 04:04:04,2019-02-23 06:37:47,company,organization
70350,ParallelM,company,/organization/parallel-machines,parallelm.com,http://www.parallelm.com,USA,CA,SF Bay Area,Sunnyvale,"1290 Oakmead Parkway, Suite #301",...,https://www.linkedin.com/company/parallel-mach...,https://www.crunchbase.com/organization/parall...,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/parallelm_ai,ParallelM - The MLOps,caf033b7-c50a-d024-21af-c37e62b49d2e,2015-11-04 18:19:05,2019-01-10 02:38:14,company,organization
71566,Puttmee by Kausoft GmbH,company,/organ

In [4]:
ai_df= ai_df.reset_index(drop=True)
ai_df

,company_name,roles,permalink,domain,homepage_url,country_code,state_code,region,city,address,...,linkedin_url,cb_url,logo_url,twitter_url,aliases,uuid,created_at,updated_at,primary_role,type
0,Ubiquity.ai,company,/organization/ubiquity-ai,ubiquity.ai,http://ubiquity.ai,NaN,NaN,NaN,NaN,NaN,...,https://www.linkedin.com/company/ubiquity.ai/,https://www.crunchbase.com/organization/ubiqui...,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,aaa64a6f-2e07-4af1-b81e-dfbd362bd82f,2018-06-14 11:25:44,2018-10-27 00:21:13,company,organization
1,Viralheat,"company,investor",/organization/viralheat,viralheat.com,http://www.viralheat.com,USA,CA,SF Bay Area,San Mateo,1510 Fashion Island Blvd. Ste. 260,...,https://www.linkedin.com/company/viralheat/,https://www.crunchbase.com/organization/viralheat,https://crunchbase-production-res.cloudinary.c...,http://twitter.com/viralheat,NaN,500ae0bc-dcb0-9d61-f228-92289c56a8ef,2009-05-25 21:49:10,2018-12-11 11:26:45,company,organization
2,DealStart,company,/organization/dealstart,dealstart.com,https://dealstart.com,USA,NY,New York City,New York,"79 Madison Avenue, 7th Floor",...,https://www.linkedin.com/company/dealstart,https://www.crunchbase.com/organization/dealstart,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/dealstart,NaN,a7fe8d37-4ab6-71e9-4a7b-3d0bc4569eeb,2014-03-11 00:39:35,2019-01-19 03:56:17,company,organization
3,UniQreate,company,/organization/uniqreate-limited,uniqreate.com,http://www.uniqreate.com,USA,NY,New York City,New York,"353 Lexington Avenue, Suite 1605",...,https://www.linkedin.com/company/uniqreate,https://www.crunchbase.com/organization/uniqre...,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/Uni_Qreate,"UQ,ReQon,Xtract",b85e315c-a61c-adc1-6a78-5c57300a4361,2015-09-22 07:43:15,2019-01-28 04:31:27,company,organization
4,Really,company,/organization/really-really-inc,really.ai,https://really.ai/,USA,CA,SF Bay Area,Palo Alto,950 Page Mill Road,...,https://www.linkedin.com/company-beta/17978433/,https://www.crunchbase.com/organization/really...,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/reallyreallyio,NaN,0441a298-919d-8999-2977-88a7b144ff06,2017-03-08 04:33:08,2018-10-26 23:19:07,company,organization
5,Flyreel,company,/organization/flyreel,flyreel.com,http://flyreel.com,USA,CO,Denver,Denver,3000 Lawrence Street,...,https://www.linkedin.com/company/2933039,https://www.crunchbase.com/organization/flyreel,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,9dad2e54-1b28-ddbb-9d5f-e4b1263deccc,2015-08-25 08:24:47,2018-10-26 22:17:06,company,organization
6,DISCERN,company,/organization/discern,discern.com,http://www.discern.com,USA,CA,SF Bay Area,San Mateo,493 South El Camino Real,...,https://www.linkedin.com/company/discern,https://www.crunchbase.com/organization/discern,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/DISCERN_Data,NaN,491d642a-6899-fd3c-5a85-d2851ee8c89e,2012-12-14 16:07:20,2019-01-25 11:55:18,company,organization
7,Polte Corporation,company,/organization/polte,polte.com,https://www.polte.com/,USA,TX,Dallas,Richardson,801 E. Campbell Road\nSuite 580\n,...,https://www.linkedin.com/company/polte/,https://www.crunchbase.com/organization/polte,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/PoLTECorp,Invisitrack,308560bf-494f-4ab4-8ce3-b7164450fa62,2018-04-09 04:04:04,2019-02-23 06:37:47,company,organization
8,ParallelM,company,/organization/parallel-machines,parallelm.com,http://www.parallelm.com,USA,CA,SF Bay Area,Sunnyvale,"1290 Oakmead Parkway, Suite #301",...,https://www.linkedin.com/company/parallel-mach...,https://www.crunchbase.com/organization/parall...,https://crunchbase-production-res.cloudinary.c...,https://twitter.com/parallelm_ai,ParallelM - The MLOps,caf033b7-c50a-d024-21af-c37e62b49d2e,2015-11-04 18:19:05,2019-01-10 02:38:14,company,organization
9,Puttmee by Kausoft GmbH,company,/organization/kausoft-gmbh,puttmee.com,http://

In [5]:
# location given here 
location = "philadephia"

# defining a params dict for the parameters to be sent to the API 
PARAMS = {'address':location} 
for i in range(len(ai_df)):
    permalink = ai_df['permalink'][i].split("/")[2]
    newpath = '/Users/ksun/downloads/startup-data/ai-organizations/' + permalink
    if not os.path.exists(newpath):
        print("create new folder", newpath)
        os.makedirs(newpath)
    
        url = 'https://api.crunchbase.com/v3.1/organizations/' + permalink+'?relationships=news&user_key=662e263576fe3e4ea5991edfbcfb9883'
        response_json = requests.get(url, PARAMS).json()

        for n in response_json['data']['relationships']['news']['items']:
            try:
                uri=n['properties']['url']
                opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor)
                opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.77 Safari/535.7')]
                html =  opener.open(uri).read().decode('utf-8')
                readable_article = Document(html).summary()
                soup = BeautifulSoup(readable_article, "lxml")
                text = soup.get_text()

                name_of_file = uri.replace(".", "-").replace(":", "-").replace("/", "-")
                path = newpath + "/"

                completeName = os.path.join(path, name_of_file+".txt")  


                file1 = open(completeName, "w")

                file1.write(text)

                file1.close()
            except:
                pass
            
    if len(os.listdir(newpath)) == 0:
        print("empty", newpath)
    
        url = 'https://api.crunchbase.com/v3.1/organizations/' + permalink+'?relationships=news&user_key=662e263576fe3e4ea5991edfbcfb9883'
        response_json = requests.get(url, PARAMS).json()

        for n in response_json['data']['relationships']['news']['items']:
            try:
                uri=n['properties']['url']
                opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor)
                opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.77 Safari/535.7')]
                html =  opener.open(uri).read().decode('utf-8')
                readable_article = Document(html).summary()
                soup = BeautifulSoup(readable_article, "lxml")
                text = soup.get_text()

                name_of_file = uri.replace(".", "-").replace(":", "-").replace("/", "-")
                path = newpath + "/"

                completeName = os.path.join(path, name_of_file+".txt")  


                file1 = open(completeName, "w")

                file1.write(text)

                file1.close()
            except:
                pass


empty /Users/ksun/downloads/startup-data/ai-organizations/ubiquity-ai
empty /Users/ksun/downloads/startup-data/ai-organizations/dealstart
empty /Users/ksun/downloads/startup-data/ai-organizations/uniqreate-limited
empty /Users/ksun/downloads/startup-data/ai-organizations/discern
empty /Users/ksun/downloads/startup-data/ai-organizations/kausoft-gmbh
empty /Users/ksun/downloads/startup-data/ai-organizations/browsi
empty /Users/ksun/downloads/startup-data/ai-organizations/cignal-io-ltd
empty /Users/ksun/downloads/startup-data/ai-organizations/fyt-2
empty /Users/ksun/downloads/startup-data/ai-organizations/virtualapt
empty /Users/ksun/downloads/startup-data/ai-organizations/ohmybet
empty /Users/ksun/downloads/startup-data/ai-organizations/dynamic-components
empty /Users/ksun/downloads/startup-data/ai-organizations/spill-ad
empty /Users/ksun/downloads/startup-data/ai-organizations/siyea-2
empty /Users/ksun/downloads/startup-data/ai-organizations/sigmind
empty /Users/ksun/downloads/startup-d

empty /Users/ksun/downloads/startup-data/ai-organizations/xihelm
empty /Users/ksun/downloads/startup-data/ai-organizations/aphrie
empty /Users/ksun/downloads/startup-data/ai-organizations/ation-labs-corp
empty /Users/ksun/downloads/startup-data/ai-organizations/shop-you
empty /Users/ksun/downloads/startup-data/ai-organizations/arcadya-ltd
empty /Users/ksun/downloads/startup-data/ai-organizations/qsays
empty /Users/ksun/downloads/startup-data/ai-organizations/getlocal-a837
empty /Users/ksun/downloads/startup-data/ai-organizations/wind-power-lab
empty /Users/ksun/downloads/startup-data/ai-organizations/legendary-ai
empty /Users/ksun/downloads/startup-data/ai-organizations/nautilus-labs-2
empty /Users/ksun/downloads/startup-data/ai-organizations/automatter-io
empty /Users/ksun/downloads/startup-data/ai-organizations/barac
empty /Users/ksun/downloads/startup-data/ai-organizations/datasmart-llc
empty /Users/ksun/downloads/startup-data/ai-organizations/intellimize
empty /Users/ksun/downloads

empty /Users/ksun/downloads/startup-data/ai-organizations/frizbit
empty /Users/ksun/downloads/startup-data/ai-organizations/roomor-proptech
empty /Users/ksun/downloads/startup-data/ai-organizations/whipper
empty /Users/ksun/downloads/startup-data/ai-organizations/shyn
empty /Users/ksun/downloads/startup-data/ai-organizations/hostnfly
empty /Users/ksun/downloads/startup-data/ai-organizations/sereine-lab
empty /Users/ksun/downloads/startup-data/ai-organizations/crafty
empty /Users/ksun/downloads/startup-data/ai-organizations/elemetric
empty /Users/ksun/downloads/startup-data/ai-organizations/stride-software
empty /Users/ksun/downloads/startup-data/ai-organizations/astraea-inc
empty /Users/ksun/downloads/startup-data/ai-organizations/reelevant
empty /Users/ksun/downloads/startup-data/ai-organizations/apio-4
empty /Users/ksun/downloads/startup-data/ai-organizations/monsoon-credittech
empty /Users/ksun/downloads/startup-data/ai-organizations/brolly
empty /Users/ksun/downloads/startup-data/a

KeyboardInterrupt: 

Hi Kevin
Thanks again for coming in! And great job – this is an excellent first step.
Here are the next steps:
1. I went through the folders – most look good, but it’s possible there’s some missing material. Spend a little more time to perfect the downloading  code. See if you can fix whichever errors you find through the process. It might be best to make this code as smooth as possible now, before we scale to working with lots of companies.
2. I am assuming the scraping process is OK, but it’s probably helpful to take one more look at that too (i.e. for example, those few lines missing in the TXT we looked at together may happen elsewhere too, no huge deal but worth looking into it).
3. Once you’ve finalized the downloading process, let’s start with a very simple exercise on the TXT files. For each TXT of each company, write a code that counts the number of times the following keywords are mentioned: PROTOTYPE, TEST, EXPERIMENT, MINIMUM VIABLE PRODUCT, BETA, ALPHA. This is just a start, we’ll do more of this, but let’s see how this goes. You can then input this data into an Excel file (one row per company).
 
If possible, please write your Python code clearly and upload it. So I can try to take a look.
 
Please remember to submit your timesheet by Thursday night each week. Let’s try to meet week after Spring break, I am very flexible. Of course, please reach out if you have any questions at any point.
Thanks so much!
A

In [55]:
df.loc[df['permalink'] == '/organization/xotiv-technologies'].index.values.astype(int)[0]

732719

In [63]:
import shutil
rootdir = '/Users/ksun/downloads/startup-data/ai-organizations/'

ai_df['articles'] = np.empty((len(ai_df), 0)).tolist()

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if '.DS_Store' not in file:
#             if(len(ai_df[ai_df['permalink'].astype(str).str.contains('/organization/' + subdir.split("/")[-1])])==0):
#                 print('/organization/' + subdir.split("/")[-1])
#                 shutil.rmtree(subdir)
#                 break
            row_index = ai_df.loc[ai_df['permalink'] == '/organization/' + subdir.split("/")[-1]].index.values.astype(int)[0]
            file = open(subdir +  "/" + file, "r")
            ai_df['articles'][row_index].append(file.read())
            file.close()

In [69]:
# load nltk's English stopwords
stopwords = nltk.corpus.stopwords.words('english')
# load nltk's SnowballStemmer
stemmer = SnowballStemmer("english")

# Define a tokenizer and stemmer that returns the set of stems of the text
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if len(token) <= 1: # remove one character words, "a" apostraphe s.
            continue
        if token.isalpha():
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if len(token) <= 1: # remove one character words, "a" apostraphe s.
            continue
        if token.isalpha():
            filtered_tokens.append(token)
    return filtered_tokens

In [ ]:
from itertools import chain
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize

ai_df['num_prototype'] = 0
ai_df['num_test'] = 0
ai_df['num_experiment'] = 0
ai_df['num_minimum_viable_product'] = 0
ai_df['num_beta'] = 0
ai_df['num_alpha'] = 0
for i in range(len(ai_df)):
    wordlist = []
    for a in ai_df['articles'][i]:
        wordlist.extend(list(chain(*[word_tokenize(s) for s in sent_tokenize(a)])))
    counter = Counter(wordlist)
    ai_df['num_prototype'][i] = counter['prototype']
    ai_df['num_test'][i] = counter['test']
    ai_df['num_experiment'][i] = counter['experiment']
    ai_df['num_minimum_viable_product'][i] = counter['viable']
    ai_df['num_beta'][i] = counter['beta']
    ai_df['num_alpha'][i] = counter['alpha']

/Users/ksun/Downloads/venv3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/ksun/Downloads/venv3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ksun/Downloads/venv3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ksun/Downloads/venv3/lib/python3.7/site-packages/ipykernel_launcher.py:19: Se

In [2]:
ai_df = pd.read_csv('csv_export/ai_df.csv')

import requests

URL = 'https://www.google.com/search?pz=1&cf=all&ned=us&hl=en&tbm=nws&gl=us&as_q={query}&as_occt=any&as_drrb=b&as_mindate={from_month}%2F%{from_day}%2F{from_year}&as_maxdate={to_month}%2F{to_day}%2F{to_year}&tbs=cdr%3A1%2Ccd_min%3A3%2F1%2F13%2Ccd_max%3A3%2F2%2F13&authuser=0'


def run(**params):
    response = requests.get(URL.format(**params))
#     print(response.content, response.status_code)
    print(response.text)


run(query="github", from_month=1, from_day=1, from_year=16, to_month=12, to_day=31, to_year=18)

<!doctype html><html itemscope="" itemtype="http://schema.org/SearchResultsPage" lang="en"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><noscript><meta content="0;url=/search?q=github&amp;tbm=nws&amp;tbs=cdr:1,cd_min:3/1/13,cd_max:3/2/13&amp;hl=en&amp;gl=us&amp;as_drrb=b&amp;authuser=0&amp;ie=UTF-8&amp;gbv=1&amp;sei=SoihXOHKJKy60PEP7vC4oA4" http-equiv="refresh"><style>table,div,span,p{display:none}</style><div style="display:block">Please click <a href="/search?q=github&amp;tbm=nws&amp;tbs=cdr:1,cd_min:3/1/13,cd_max:3/2/13&amp;hl=en&amp;gl=us&amp;as_drrb=b&amp;authuser=0&amp;ie=UTF-8&amp;gbv=1&amp;sei=SoihXOHKJKy60PEP7vC4oA4">here</a> if you are not redirected within a few seconds.</div></noscript><title>github - Google Search</title><style>#gbar,#guser{font-size:13px;padding-top:1px !important;}#gbar{height:22px}#guser{padding-bottom:7px !important;text-align:right

In [3]:
ai_df.loc[ai_df['permalink'] == '/organization/' + "tiger-digital"].index.values.astype(int)[0]

1935

In [49]:
import shutil
rootdir = '/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets'

ai_df['tweets'] = np.empty((len(ai_df), 0)).tolist()

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if '.DS_Store' not in file:
#             if(len(ai_df[ai_df['permalink'].astype(str).str.contains('/organization/' + subdir.split("/")[-1])])==0):
#                 print('/organization/' + subdir.split("/")[-1])
#                 shutil.rmtree(subdir)
#                 break
            row_index = ai_df.loc[ai_df['permalink'] == '/organization/' + file.split("/")[-1][0:-4]].index.values.astype(int)[0]
#             file = open(subdir +  "/" + file, "r")
            print(subdir +  "/" + file)
            temp_tweet_df = pd.read_csv(subdir +  "/" + file, error_bad_lines=False)
            for j in range(len(temp_tweet_df)):
                try:
                    ai_df['tweets'][row_index].append([str(temp_tweet_df['text'][j])])
                except:
                    continue

/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/bayonet.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/digital-startups.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/cervest.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/intogral.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/tuple-technologies.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/3one4.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/retinai-medical.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/tenderlake.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/pilot-automotive-labs.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/kudi.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/missinglink-ai.csv
/Users/ksun/downl

b'Skipping line 66: expected 11 fields, saw 12\nSkipping line 73: expected 11 fields, saw 12\n'
b'Skipping line 312: expected 11 fields, saw 12\n'
b'Skipping line 34: expected 11 fields, saw 12\n'
b'Skipping line 38: expected 11 fields, saw 12\n'
b'Skipping line 45: expected 11 fields, saw 12\n'
b'Skipping line 24: expected 11 fields, saw 12\nSkipping line 84: expected 11 fields, saw 12\nSkipping line 92: expected 11 fields, saw 12\nSkipping line 116: expected 11 fields, saw 12\n'
b'Skipping line 4531: expected 11 fields, saw 12\nSkipping line 5119: expected 11 fields, saw 12\nSkipping line 5222: expected 11 fields, saw 13\nSkipping line 7110: expected 11 fields, saw 13\nSkipping line 7258: expected 11 fields, saw 12\n'


/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/workfit.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/understandbetter.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/pwave-technologies.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/forecasthq.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/polanyio.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/basiq.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/intellimize.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/riseml.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/diaalog.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/etsimo-ltd.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/polte.csv
/Users/ksun/downloads/startup-data/Twit

b'Skipping line 5: expected 11 fields, saw 12\nSkipping line 16: expected 11 fields, saw 13\nSkipping line 43: expected 11 fields, saw 12\nSkipping line 63: expected 11 fields, saw 12\nSkipping line 85: expected 11 fields, saw 13\nSkipping line 93: expected 11 fields, saw 13\nSkipping line 96: expected 11 fields, saw 14\nSkipping line 98: expected 11 fields, saw 12\nSkipping line 99: expected 11 fields, saw 12\nSkipping line 101: expected 11 fields, saw 12\nSkipping line 106: expected 11 fields, saw 12\nSkipping line 107: expected 11 fields, saw 12\nSkipping line 123: expected 11 fields, saw 13\nSkipping line 127: expected 11 fields, saw 13\nSkipping line 171: expected 11 fields, saw 12\nSkipping line 473: expected 11 fields, saw 13\nSkipping line 606: expected 11 fields, saw 12\n'
b'Skipping line 12: expected 11 fields, saw 12\nSkipping line 33: expected 11 fields, saw 12\nSkipping line 378: expected 11 fields, saw 12\nSkipping line 380: expected 11 fields, saw 12\n'
b'Skipping line 3

/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/lzine.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/superflex.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/explosion-ai.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/photolemur.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/mjr-analytics.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/botfuel.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/latently.csv


...

b'Skipping line 47: expected 11 fields, saw 12\n'
b'Skipping line 18: expected 11 fields, saw 12\nSkipping line 44: expected 11 fields, saw 12\nSkipping line 71: expected 11 fields, saw 12\nSkipping line 78: expected 11 fields, saw 13\n'
b'Skipping line 855: expected 11 fields, saw 16\nSkipping line 1504: expected 11 fields, saw 12\nSkipping line 1922: expected 11 fields, saw 12\nSkipping line 2785: expected 11 fields, saw 12\nSkipping line 3044: expected 11 fields, saw 12\n'
b'Skipping line 29: expected 11 fields, saw 12\nSkipping line 32: expected 11 fields, saw 12\nSkipping line 54: expected 11 fields, saw 12\nSkipping line 59: expected 11 fields, saw 12\nSkipping line 85: expected 11 fields, saw 12\nSkipping line 244: expected 11 fields, saw 12\nSkipping line 321: expected 11 fields, saw 13\nSkipping line 326: expected 11 fields, saw 13\nSkipping line 346: expected 11 fields, saw 12\nSkipping line 350: expected 11 fields, saw 12\nSkipping line 357: expected 11 fields, saw 14\nSkipp

/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/olio-city.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/emotian.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/exabel.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/ingrid-2.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/cerebras-systems.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/serdoo.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/bottomless.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/letz-do.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/nubix.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/thunderbolt-pipeline.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/taste-4.csv
/Users/ksun/downloads/startup-data/Twitte

b'Skipping line 362: expected 11 fields, saw 13\nSkipping line 364: expected 11 fields, saw 12\n'
b'Skipping line 12: expected 11 fields, saw 12\nSkipping line 14: expected 11 fields, saw 12\n'
b'Skipping line 37: expected 11 fields, saw 12\n'
b'Skipping line 34: expected 11 fields, saw 13\n'
b'Skipping line 41: expected 11 fields, saw 13\nSkipping line 74: expected 11 fields, saw 12\nSkipping line 78: expected 11 fields, saw 13\n'
b'Skipping line 4: expected 11 fields, saw 12\nSkipping line 32: expected 11 fields, saw 12\nSkipping line 43: expected 11 fields, saw 12\n'
b'Skipping line 15: expected 11 fields, saw 12\nSkipping line 53: expected 11 fields, saw 13\n'
b'Skipping line 5: expected 11 fields, saw 12\n'
b'Skipping line 22: expected 11 fields, saw 12\nSkipping line 95: expected 11 fields, saw 13\nSkipping line 108: expected 11 fields, saw 12\n'
b'Skipping line 3: expected 11 fields, saw 12\nSkipping line 22: expected 11 fields, saw 12\n'
b'Skipping line 163: expected 11 fields,

/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/weave-labs.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/raccoon-technologies-inc.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/medmee.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/acutronic-robotics.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/amperity.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/sana-2.csv
/Users/ksun/downloads/startup-data/Twitter-Get-Old-Tweets-Scraper/tweets/flashrecruit.csv


b'Skipping line 44: expected 12 fields, saw 13\n'
b'Skipping line 10: expected 11 fields, saw 12\nSkipping line 24: expected 11 fields, saw 12\nSkipping line 30: expected 11 fields, saw 13\nSkipping line 32: expected 11 fields, saw 12\nSkipping line 76: expected 11 fields, saw 13\n'
b'Skipping line 7: expected 11 fields, saw 12\nSkipping line 37: expected 11 fields, saw 12\nSkipping line 52: expected 11 fields, saw 13\nSkipping line 65: expected 11 fields, saw 12\nSkipping line 70: expected 11 fields, saw 12\n'


In [50]:
ai_df

,company_name,roles,permalink,domain,homepage_url,country_code,state_code,region,city,address,...,primary_role,type,articles,num_prototype,num_test,num_experiment,num_minimum_viable_product,num_beta,num_alpha,tweets
0,Ubiquity.ai,company,/organization/ubiquity-ai,ubiquity.ai,http://ubiquity.ai,NaN,NaN,NaN,NaN,NaN,...,company,organization,[],0,0,0,0,0,0,[]
1,Viralheat,"company,investor",/organization/viralheat,viralheat.com,http://www.viralheat.com,USA,CA,SF Bay Area,San Mateo,1510 Fashion Island Blvd. Ste. 260,...,company,organization,[('http://adexchanger.com/digital-marketing-2/...,0,3,0,0,0,0,[]
2,DealStart,company,/organization/dealstart,dealstart.com,https://dealstart.com,USA,NY,New York City,New York,"79 Madison Avenue, 7th Floor",...,company,organization,[],0,0,0,0,0,0,[[@ mandrillapp There seem to be issues with y...
3,UniQreate,company,/organization/uniqreate-limited,uniqreate.com,http://www.uniqreate.com,USA,NY,New York City,New York,"353 Lexington Avenue, Suite 1605",...,company,organization,[],0,0,0,0,0,0,"[[3 wk roadshow ends. 43 meetings, 27 Enterpri..."
4,Really,company,/organization/really-really-inc,really.ai,https://really.ai/,USA,CA,SF Bay Area,Palo Alto,950 Page Mill Road,...,company,organization,[('https://www.bloomberg.com/view/articles/201...,0,0,0,0,0,0,[[Nice to see some focus on why early detectio...
5,Flyreel,company,/organization/flyreel,flyreel.com,http://flyreel.com,USA,CO,Denver,Denver,3000 Lawrence Street,...,company,organization,[('https://www.prnewswire.com/news-releases/mi...,0,0,0,0,0,0,[]
6,DISCERN,company,/organization/discern,discern.com,http://www.discern.com,USA,CA,SF Bay Area,San Mateo,493 South El Camino Real,...,company,organization,[],0,0,0,0,0,0,[[@ DISCERN_data's Rick Church Weighs in on # ...
7,Polte Corporation,company,/organization/polte,polte.com,https://www.polte.com/,USA,TX,Dallas,Richardson,801 E. Campbell Road\nSuite 580\n,...,company,organization,[('https://techcrunch.com/2018/09/05/polte-let...,0,3,0,0,0,0,"[[Congratulations and best of luck!], [What's ..."
8,ParallelM,company,/organization/parallel-machines,parallelm.com,http://www.parallelm.com,USA,CA,SF Bay Area,Sunnyvale,"1290 Oakmead Parkway, Suite #301",...,company,organization,[('https://www.businesswire.com/news/home/2019...,0,0,0,0,0,0,[[Don't miss our # CEO @ SivanMetzger as he di...
9,Puttmee by Kausoft GmbH,company,/organization/kausoft-gmbh,puttmee.com,http://puttmee.com,DEU,NaN,Cologne,Cologne,Wiedstr. 8,...,company,organization,[],0,0,0,0,0,0,[[NEW IN V2.7! Choose the ENTRY SPEED of your ...


In [51]:
ai_df['tweets'][265]

[['# FinTech 2.0: How a programmatic architecture for financial services is going to truly disrupt financial systems http://bit.ly/2KU7Ebw via @ psmol'],
 ['La Asociación @ fintechmx está lista para trabajar de la mano con la nueva administración para impulsar el desarrollo del sector financiero en México http://bit.ly/2ITpPMv # FinTechMX'],
 ['¡Únete al equipo de @ BayonetIO! Checa nuestra vacante de # FullStackDeveloper y aplica http://bit.ly/2JWXbPr # EmpleosTI # CDMX # FinTechMXpic.twitter.com/tbQSvLWad7'],
 ['Muchas gracias!'],
 ["Join the @ BayonetIO team! We're looking for a Full-Stack Developer in Mexico City. Learn more and apply: http://bit.ly/2JWXbPr # EmpleosTI # CDMX # FinTechMXpic.twitter.com/jxDlq24NcP"],
 ['What if, instead of bringing more attention to the marginalization of women in # FinTech, we chose to normalize women being able to participate and thrive in this field? http://bit.ly/2JYvWEc vía @ Forbes @ Rachel_Ann_C # WomenInTech'],
 ['Si antes no lo era, inverti